#**Respiratory Disease Detection with Convolutional Neural Networks**
Project mentor: Angie Liu

Rena Bi <rbi4@jh.edu>, Sydney Friedel <sfriede5@jh.edu>, Varun Harish <vharish1@jh.edu>

https://github.com/renabi30/ml-lung-scans-project.git

# **Outline and Deliverables** 


### **Uncompleted Deliverables**
1. (Would Like to Complete #2) Develop an additional model for semantic segmentation of the chest X-rays using a U-Net model, to identify particular regions in the image that indicate the presence of disease.

  Unfortunately, with the limited time and choice to expand on the interpretability aspect of our model, we were not able to accomplish this deliverable. However, we feel going deeper into the model's interpretability is extremely useful and relevant to our project's theme of incorporating Human-Centered ML aspects of fairness and bias.


### **Completed Deliverables**
1. (Must Complete #1): Train a CNN to take a chest x-ray image as input and output probability predictions for each of the 4 diagnoses: COVID-19, Tuberculosis, Pneumonia, or Normal.  

  We discuss training our baseline CNN and other CNN models in [Baselines](#baselines) and [Methods](#methods) below.

2. (Must Complete #2) Compare the performance of several combinations of hyperparameters and architectures to construct the most optimal/accurate CNN for this classification task.

   We discuss hyperparameter tuning and performance comparison in [CNN Model Evaluations](#comparing) and [Hyperparameter Tuning of Best Model](#hyperparametertuning)

3. (Must Complete #3) Effectively pre-process and combine the two image datasets, so our model can effectively train on useful, high-quality data.

  We discuss our dataset pre-processing in [Dataset](#data) and [Pre-Processing](#preprocess).

4. (Expect to Complete #1) Analyze the features learned by the best CNN to further analyze the model and its reasoning behind its predictions.

     We discuss feature map visualizations, along with explanations/analyses, in [Visualizing Best Model Feature Maps](#featuremaps).

5. (Expect to Complete #2) Determine the best scoring system for the model based on our Human-Centered ML lectures, e.g. whether to prioritize recall more than precision, etc., and ensuring that our model's evaluation system is fair.

     We discuss model evaluation and scoring systems in [Experimental Setup](#evalfunction).

6. (Expect to Complete #2) Compare the performance of our best CNN with other image classification techniques, such as an SVM or decision tree, after using a CNN as the base model/feature extractor.

     We discuss comparing our best CNN classifier to SVM and Decision Tree Classifiers in [Comparing Best CNN to Classical Models](#classicalmodels).

7. (Would Like to Complete #3) Experiment with image data augmentation (e.g. reflecting images) and histogram equalization to see how transformations of the X-ray images affect model performance.

     We discuss experimenting with histogram equalization, and the resulting effects on our best model's performance, in [Histogram Equalization](#histeq). Also, data augmentation is performed on minority classes as a form of oversampling in [Performing Data Augmentation and Expanding Dataset](#expanddata).

8. (Would Like to Complete #1) Expand dataset to include images of chest X-rays labeled with a larger variety of respiratory diseases, to see if our best model can adapt to a more diverse collection of labels.

     We discuss expanding our dataset to include different types of diseases, in [Performing Data Augmentation and Expanding Dataset](#expanddata).


### **Additional Deliverables**
1. We spent significant time producing visualizations of the model's innerworkings and performance, such as GradCAM heatmaps, feature maps, and AUC-ROC curves. This was important for exploring the interpretability of our model in [Interpretability of Best CNN Model](#interpret).

# **Preliminaries**

## **What problem were you trying to solve or understand?**

The problem we're trying to solve is predicting the presence of a respiratory disease in a given patient's chest X-Ray scan based on the image itself. Our plan is to solve this problem using a supervised machine learning classifcation method, Convolutional Neural Networks (CNN). Our hope is that a CNN will be able to recognize patterns and anomalies in the chest X-rays that allow it to detect the presence of a disease or no disease at all.

**What are the real-world implications of this data and task?**
The aim is to create a classifcation model that can automatically detect the presence of a disease (Tuberculosis, Pneumonia, Covid-19, or No Disease) via a patient's gray-scale chest X-Ray.

By automating disease detection with accurate, reliable models, medical professionals will be assisted, and effective diagnoses can be delivered sooner. Early detection can help to reduce the negative effects on physical health associated with such infections and even help prevent a patient's death.

The harmful effects of COVID-19 are well-known, so the importance of early disease detection cannot be understated. In addition, pneumonia and tuberculosis are harmful infections that can greatly damage one's lungs and even be life-threatening if not treated in a timely manner. In the United States in 2020, there were 7,174 cases of tuberculosis per 100,000 people 47,601 deaths caused by pneumonia, so these are significant diseases (CDC).

**How is this problem similar to others we’ve seen in lectures, breakouts, and homeworks?**

This problem relates to lecture discussions and homework problems on supervised classification techniques, which CNNs fall under. In particular, we've discussed how CNNs work in class and had a homework question concerning convolutional filter maps. 

Relating to the Human-Centered ML unit, we had many in-lecture discussions and homework problems about fair evaluation methods and their importance when using machine learning for medical purposes. We had a homework problem concerning the F-Beta metric for classifcation model evaluation, and how to choose beta to prioirtize recall and precision in a manner that best suited the problem. Our problem of repsiratory disease detection is strongly related, as these diseases we're trying to predict are fairly uncommon and can have drastic physical effects.

**What makes this problem unique?**

One unique aspect of this problem is its cuurent relevance. One of the diseases being detected within our image dataset is Covid-19. Of course, this disease is still very prevalent and a matter of concern today. Being able to create machine leanring models to assist with Covid-19 models is incredibly valuable to medical professionals today.

On a more general note, as described above, this problem is unique due to its real-life applications and ability to assist medical professionals. 

**What ethical implications does this problem have?**

For this problem of predicting the presence of uncommon, harmful, and even fatal diseases, the model's accuracy, recall, and precision are extremely important metrics consider and ensure good performance. If an inaccurate model is deployed in the real world, this can lead to inaccurate diagnoses, which is extremely harmful. Thus, there are severe ethical implications if the model is not thoroughly tested and evaluated.

If a model used for predicting the presence of any of these diseases has low recall and high False Negative rate, there are severe ethical implications. If a patient is sick, but the model incorrectly predicts no disease is present, the patient may not receive a proper diagnosis. Without proper medical treatment, this can lead to terrible health effects or even death.

If the model has low precision and high False Positive rate, there are ethical implications. If a patient is healthy, but the model incorrectly predicts some disease is present, the patient may experience unnecessary panic and stress, and/or they may endure unneeded medical bills for further tests and treatment.

Thus, both recall and precision are important metrics to consider in model evaluations for our problem.

**Installing some necessary packages...**

In [ ]:
# Load your data and print 2-3 examples
# upgrade pip
!pip install --upgrade pip
# install necessary package
!pip install opencv-python
!pip install keras_tuner

In [ ]:
# import libraries
import cv2 # for reading images
from PIL import Image # for displaying images
import os
import numpy as np


<a name = 'data'></a>
## **Dataset(s)**

**Describe the dataset(s) you used.**
We merged two datasets of Portable Network Graphic (PNG) image files. These are gray-scale, RGB images of chest X-ray scans, where each scan has a corresponding respiratory disease label, or the patient is healthy so they are labeled 'Normal'. Both datasets soruces from various public, accessible medical data.

The first dataset contains 3,616 COVID-19 positive cases,
10,192 Normal, and 1,345 Viral Pneumonia chest X-ray scans, with resolution 299x299 pixels

The second dataset contains 3,500 images labeled
'Normal’ and 700 images labeled ’Tuberculosis,' with resolution 512x512 pixels. 

We chose these datasets because they consist of high-quality chest X-ray scans with labeled diseases present (or no disease). Across the two datasets, we have data four 4 categories, making our classifcation task multiclass.

The two datset sources are listed here:

First dataset: https://www.kaggle.com/datasets/tawsifurrahman/covid19-radiography-database?search=lung+opacity

Second dataset: https://www.kaggle.com/datasets/tawsifurrahman/tuberculosis-tb-chest-xray-dataset



<a name = 'preprocess'></a>
## **Pre-processing**

Given that this is an image classification task, the features for our model/task are the arrays of RGB pixel data from the images of Chest X-ray scans. Thus, each image is reshaped into a 2-dimensional array with 3 color channels. We later reshape each image to all be the same dimension (299x299)

The first step is to load the image data from the five folders (Normal 1, Normal 2, Tuberculosis, Viral Pnemonia, and COVID-19). 

In [ ]:
!unzip lung-scans-dataset

# get the different image sets, original dimensions are indicated

covid_path="lung-scans-dataset/covid" # 299 x 299 
normal1_path="lung-scans-dataset/normal-1" # 299 x 299
normal2_path="lung-scans-dataset/normal-2" # 512 x 512
tuber_path="lung-scans-dataset/tuberculosis" # 512 x 512
pneum_path="lung-scans-dataset/viral-pneumonia" # 299 x 299

img_D = 299 # image dimension

For each categorical folder, we iterate through the images, resize each to 299x299 pixels, convert the image data to a Numpy array, and append the array to a list of examples for that specific category. Simulatenously, we construct the list of corresponding labels for all images, following the coding scheme:

>Normal: 0

>COVID-19: 1

>Tuberculosis: 2

>Pneumonia: 3 

 Since the two merged datasets consisted of images of different resolutions/sizes, we needed to resize all images to the same dimensions.


There is a tradeoff between decreasing and increasing the dimensions of the images that the CNNs (which use fixed-size input) learn on. Compressing the images too much could create a loss of information, which can hinder the model's inability to detect patterns and extract useful features for classification. However, making the images too large greatly increases the training time complexity, which makes training impossible due to time constraints. 299x299 pixels was a good middle-ground for image dimension for both preserving information and having a reasonable training time.

We used Python's OpenCV library to resize the images, which it does so via pixel resampling.


Since we gathered data from two different datasets, each of which had chest X-rays classified as Normal, we first needed to eliminate any duplicate images between those two sets.

In [ ]:
# read in normal images and store in X, y arrays
normal_X = []
normal_y = [] #0

for img in os.listdir(normal1_path):
    image_path=normal1_path+"/"+img
    img_arr=cv2.resize(cv2.imread(image_path),(img_D,img_D))
    normal_X.append(img_arr)

for img in os.listdir(normal2_path):
    image_path=normal2_path+"/"+img
    img_arr=cv2.resize(cv2.imread(image_path),(img_D,img_D))
    normal_X.append(img_arr)

normal_X = np.unique(normal_X, axis=0) # numpy array, removing duplicates

# add labels for normal images
for x in normal_X:
    normal_y.append(0)

In [ ]:
# read in covid images and store in X, y arrays
covid_X = []
covid_y = [] #1

for img in os.listdir(covid_path):
    image_path=covid_path+"/"+img
    img_arr=cv2.resize(cv2.imread(image_path),(img_D,img_D))
    covid_X.append(img_arr)
    covid_y.append(1)

In [ ]:
# convert tuberculosis images into arrays
tuber_X = []
tuber_y = [] #2

for img in os.listdir(tuber_path):
    image_path=tuber_path+"/"+img
    img_arr=cv2.resize(cv2.imread(image_path),(img_D,img_D))
    tuber_X.append(img_arr)
    tuber_y.append(2)

In [ ]:
# convert pneumonia images into arrays
pneum_X = []
pneum_y = [] #3

for img in os.listdir(pneum_path):
    image_path=pneum_path+"/"+img
    img_arr=cv2.resize(cv2.imread(image_path),(img_D,img_D))
    pneum_X.append(img_arr)
    pneum_y.append(3)

Next, we combined all data from the four categories into a list of image data arrays, X, and a corresponding vector of labels, y. Then, we randomly shuffled the data and performed an 80-10-10 train-validation-test datasplit. We chose 80-10-10 as our split, as it is a popular choice for how to split the data into these three sets.

In [ ]:
# combine all X data and y data
X = np.concatenate((covid_X, normal_X, tuber_X, pneum_X), axis=0)
y = np.concatenate((covid_y, normal_y, tuber_y, pneum_y), axis=0)

In [ ]:
# randomize all data
from sklearn.utils import shuffle
X, y = shuffle(X, y, random_state=0)

In [ ]:
# create train/test/val split for each category and label data
from sklearn.model_selection import train_test_split
# creates a train set made of 80% of the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
# splits the previous test set into test and val evenly, so each is 10% of the data
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.50)

These are a few examples of the chest X-ray images once preprocessing is complete:

In [ ]:
# 3 examples of images from dataset
PIL_image_covid = Image.fromarray(np.uint8(covid_X[0]))
display(PIL_image_covid)

PIL_image_tuber = Image.fromarray(np.uint8(tuber_X[0]))
display(PIL_image_tuber)

PIL_image_normal = Image.fromarray(np.uint8(normal_X[0]))
display(PIL_image_normal)

![picture](https://drive.google.com/uc?export=view&id=1K6H3VHMp2GshbTmNHYXBEUCwK931gn-p) **Covid-19 Present**

![picture](https://drive.google.com/uc?export=view&id=1VQtg1rNneVUMeJytDQjMJKif3_y4wPig)**No Disease Present**

![picture](https://drive.google.com/uc?export=view&id=1ct_URe-WDvYJwBaKf3C-Mpbnk8M090gn)**Tuberculosis Present**



Here is the distribution of the four categories for all of our data. We see that the classes are very imbalanced, with most of the images labeled as Normal. We will correct this imbalance when we train our models later.


![picture](https://drive.google.com/uc?export=view&id=1j9FCb4-f4VrvdKmPaLB7ESSQ7K5dXb_t)

Based on our research, there are certain patterns/anomalies in chest X-rays when a person has one of the three respiratory diseases we're trying to detect:

>> Tuberculosis: Patterns of congestion/infection tend to be towards the center of the chest/lungs, appearing on the sides of the spine.

>> Covid-19: Pateints with COVID often have hazy areas of increased density throughout the lungs, though it should be noted that this pattern is not entirely sufficient for a diagnosis

>> Pneumonia: Patterns of infection look like white spots (infiltrates) throughout the lungs.

>> Normal: Ideally, healthy, uninfected lungs have no patterns/anomalies.

Our goal is to create a model that can detect these patterns by extracting useful features for learning. In our model interpretability analysis, we will see some heatmap visualizations of where the CNN is focusing/attending to the most. This will allow us to interpret what sort of patterns/areas/features the model is trying to detect in predicting the presence of different diseases.

# **Models and Evaluation**

<a name = 'evalfunction'></a>
## **Experimental Setup**
**How did you evaluate your methods? Why is that a reasonable evaluation metric for the task?**

We evaluated our models by comparing prediction accuracies (true label versus predicted label) on the held-out test dataset. This is reasonable for our image classification task since we weighted the examples to account for the classes being unbalanced. It's a good metric for comparing how good each model is at making the correct predictions, regardless of class. We also compared plots of training and validation data accuracies and losses as a function of number of epochs to visualize the model's learning curve and its stability.

In addition, we compared confusion matrices, precision, recall, and F-beta scores for each of the models. The confusion matrices help us to determine which diseases are particularly challenging for the model to correctly classify. In a classifcation task aimed at predicting the presence of rare diseases in patients, preciscion and recall are extremely important metrics. A high False Negative rate (low recall) has severe real-world implications, as sick patients may not receive a proper diagnosis. In additon, the model's False Positive rate (related to precision) also wants to be minimized, as we want the model to be confident when it predicts a disease's presence, so as to mitigate panic and unnecessary medical costs. Thus, comparing average precision and recall, as well as precision and recall for each class, is an important evaluation metric. 

Lastly, the F-Beta score is a good metric for comparing each model's overall ability to have good precision and recall. We set Beta = 2 to prioritze recall slightly more over precision, as there are more severe implications of low recall over low precision for our rare disease detection task.

**What did you use for your loss function to train your models? Did you try multiple loss functions? Why or why not?**

We used the Sparse Categorical Cross-Entropy Function as our loss function. The cross-entropy loss function is most common for classification models, especially deep learning models. Sparse Categorical Cross-Entropy Loss is TensorFlow's loss function for multiclass classifcation problems, where the Y labels are encoded as Yi = {1, 2, 3, ...} rather than a one-hot encoding.

We did not try multiple loss functions, as cross-entropy loss is standard for doing multi-class image classifcation with CNNs.

**How did you split your data into train and test sets? Why?**

We combined all labeled images for all four categories (Normal, Tuberculosis, Pneumonia, and Covid-19), randomly shuffled the combined dataset, and performed an 80-10-10 train-validation-test data split using the train_test_split function. This is a standard way of dividing the data into train, validation, and test sets for model fitting and evaluation.


### **Functions for Accuracy Plots, Confusion Matrices, F-Beta Score Computations**
These are the functions we called on each CNN model after training it and gathering its predictions on the test set. Deeper explanations of each function are below.


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report,confusion_matrix, precision_score, recall_score
import seaborn as sns
import pandas as pd

#Given a model's training history from when it was fit to the training and validation data,
#plot_accuracy plots the training and validation accuracy as a function of number of epochs.
#This allows us to visualize the model's learning curve, its stability, and when the model began to overfit, if it did.
def plot_accuracy(history):
  plt.plot(history.history['acc'], label='accuracy')
  plt.plot(history.history['val_acc'], label = 'validation accuracy')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.ylim([0, 1])
  plt.legend(loc='lower right')

#Given a model's training history from when it was fit to the training and validation data,
#plot_accuracy plots the training and validation loss as a function of number of epochs.
#This allows us to visualize the model's learning curve, its stability, and when the model began to overfit, if it did.
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label = 'validation loss')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend(loc='lower right')

#Given the list of test data labels, y_test, and the model's predictions on the test data,
#plot_confusion_matrix plots the multiclass confusion matrix for the model on the test data.
#This allows us to calculate False Positive/Negatives, True Positives/Negatives, and visualize
#which of the classes the model was most and least confused with in terms of being able to classify it correctly.
def plot_confusion_matrix(y_test, predictions):
  cm = pd.DataFrame(data=confusion_matrix(y_test, predictions))
  sns.heatmap(cm,annot=True,fmt="d")

#Given the list of test data labels, y_test, the model's predictions on the test data, and a choice the F-beta
# score parameter, beta, generate_classification_report uses built in functions to print the model's recall, precision, and accuracy
#on each class, as well as other metrics. In addition, it computes the model's average recall and precision between all classes, and uses that to compute
#the F-beta score.
def generate_classification_report(y_test, predictions, beta):
  print(classification_report(y_true=y_test,y_pred=predictions,labels = [0,1,2,3], target_names =['NORMAL','COVID', 'TUBERCULOSIS', 'PNEUMONIA']))
  cm = confusion_matrix(y_test, predictions,labels = [0,1,2,3])
  recall = np.mean(np.diag(cm) / np.sum(cm, axis = 1))
  precision = np.mean(np.diag(cm) / np.sum(cm, axis = 0))
  print('average recall: ', recall, ', average precision: ', precision)
  F_beta_score  = ((1 + np.square(beta)) * precision * recall) / (np.square(beta) * precision * recall)
  print('F-Beta Score: ', F_beta_score)

<a name = 'baselines'></a>
## **Baselines** 

**What baselines did you compare against? Why are these reasonable?**

We used a simple convolutional neural network (Model 0) with 3 convolutional layers and 2 max pooling layers as our baseline model for image classifcation. This is our shallowest model that serves as a reasonable baseline for comparing to more complex CNN architectures in model evaluations. 

We chose CNNs as our main classification model, as these are commonly used deep learning models for image classification problems today. In our research, we found for pretty much all medical imaging classifcation tasks, CNNs were the main method used. Toay, CNNs are most popular for image classifcation tasks because, compared to classical models like SVMs, CNNs automatically extract features from images using local spatial information. This recognition of patterns that allow the model to detect useful features and classify images well is achevieved through the use of convolution kernels and filter maps. This process of convolution and building translational invariance to detect features mimics how the human eye detects edges and objects.


<a name = 'methods'></a>
## **Methods**

**What methods did you choose? Why did you choose them?**
We have implemented 4 different CNN architectures (including the baseline architecture described above) to perform image classifcation. The three other architectures are a deeper CNN (Model 1), an implementation of the AlexNet architecture (Model 2), and a model that does transfer learning with a pre-trained VGG-16 architecture (Model 3). 

We implemented Model 1 to compare the effect of adding more hidden layers and dropout layers and the resulting change in model performance compared the the baseline model. We implemented Models 2 and 3 since AlexNet and VGG-16 are extremely popular and high-performing CNN architectures. We wanted to see how well these architectures perfomed on our dataset/specific problem. More detailed explanations of the different architectures are below.

Finally, after training and evaluating the CNN classifiers, we determined which model performed the best. Then, we compared the performance of our best model to SVM and Random Forest Image classification models. We wanted to see how other, more classical image classifcation techniques compared to just using a CNN.

**How did you train these methods, and how did you evaluate them? Why?**

We trained our models by fitting them to the labeled training data set of images and used validation sets to. To account for the unbalanced classes (there were far more images labeled 'Normal' than any disease), we weighted the 4 different categories before fitting the models to the training data. We implemented regularization early stopping to help prevent the CNN models from overfitting on the training data and learning rate reduction to help the model converge to a local optimum. 

We evaluated the four CNN models and SVM/RFC classifiers by comparing their test accuracies, class precisions and recalls, F-Beta scores with beta = 2, and confusion matrices based on test data predictions. The reasoning for these evaluation metrics was described earlier. Also, for the CNN models, we used their training histories to plot accuracy and loss as a function of number of epochs.

**Which methods were easy/difficult to implement and train? Why?**

As we expected, the deeper and more complex the CNN architectures became, the more difficult and slower the models were to train. The AlexNet and VGG-16 architectures (Models 2 and 3) took slightly longer to train than Model 1 (which wasn't as deep of a network as either model) and substantially logner than the baseline Model 0. This was a hindrance to our ability to conduct multiple runs of different models and perform significant hyperparameter tuning.

In addition, the SVM classifier took a significant amount of time to train and even required multiple attempts because of a reoccuring dead kernel problem.The time complexity required to train this classifer on such a large dataset almost made the relevant deliverable impossible. Thankfully, this became less of an issue when we performed undersampling of the majority classes and trimmed our training set size significantly.



### **Baseline CNN and Deeper Networks for Chest X-Ray Image Classification**

Here, we define the four model architectures described earlier. Note that all take input images of fixed size 299x299 pixels.As this is a multiclass classification problem, all networks' output layers have a softmax activation function. Hidden layers use ReLU activation functions.

>> Model 1/ Deeper CNN Architecture: This model has 4 convolutional-maxpooling layers, plus 3 dropout layers to encourage a more generalizable model with uncorrelated nodes. We chose 3x3 as for our kernel size, as this is a common choice for CNNs.

>> AlexNet Architecture: Alexnet won the Imagenet large-scale visual recognition challenge in 2012. The model was proposed in 2012 by Alex Krizhevsky and his colleagues. AlexNet has eight layers, consisting of five layers with a combination of max pooling followed by 2 dropout layers and 3 fully connected layers and they use Relu activation in each of these layers except the output layer. The architecture and kernel sizes are fixed since this is a predetermined structure.

>> VGG-16 Architecture and Transfer Learning:  VGG 16 was proposed by Karen Simonyan and Andrew Zisserman of the Visual Geometry Group Lab of Oxford University in 2014. This model won 1st and 2nd place in the 2014 ILSVRC challenge. VGG16 is a type of CNN (Convolutional Neural Network) that is considered to be one of the best computer vision models to date. The creators of this model evaluated the networks and increased the depth using an architecture with very small (3 × 3) convolution filters, which showed a significant improvement on the prior-art configurations. They pushed the depth to 16–19 weight layers.

>> We used a pretrained VGG-16 model, which was trained on thousands of images, then froze its early training layers so they were no longer trainable, and added our own dense layers at the end for multiclass classification.This is the process of transfer learning. In theory, it should work well since we can use a network trained on unrelated categories in a massive dataset (usually Imagenet) and apply it to our own problem because there are universal, low-level features shared between images.  The dense layers at the end help the model learn the specific features for our dataset.

>> The architectures and kernel sizes are fixed since this is a predetermined and pretrained structure.




In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,Flatten,MaxPooling2D, Dropout, GlobalMaxPool2D, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
from tensorflow.keras.applications import VGG16

img_width = img_D 
img_height = img_D

#Baseline Model 0 : very simple, 2-conv-maxpooling layer network
model0 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height,3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(4, activation = "softmax")
])

#This model uses Adam with learning rate of 0.001 as its optimizer
model0.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

#Model 1: deeper CNN with 4 convolutional-maxpooling layers, plus dropout layers to encourage a more generalizable model with uncorrelated nodes.
model1 = Sequential([
    Conv2D(16,(3,3),activation = "relu" , input_shape = (img_width,img_height,3)) ,
    MaxPooling2D(2,2),
    Conv2D(32,(3,3),activation = "relu") ,  
    MaxPooling2D(2,2),
    Conv2D(64,(3,3),activation = "relu") ,  
    MaxPooling2D(2,2),
    Conv2D(128,(3,3),activation = "relu"),  
    MaxPooling2D(2,2),
    Flatten(), 
    Dense(550,activation="relu"),     
    Dropout(0.1,seed = 2019),
    Dense(400,activation ="relu"),
    Dropout(0.3,seed = 2019),
    Dense(300,activation="relu"),
    Dropout(0.4,seed = 2019),
    Dense(200,activation ="relu"),
    Dropout(0.2,seed = 2019),
    Dense(4,activation = "softmax")   
])

#This model uses Adam with learning rate of 0.001 as its optimizer
model1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics = ['acc'])

#Model 2: Implementation of the AlexNet architecture, which has 5 convolution-maxpooling layers, batch normalization layers - 
# which encourage smoother gradients, faster training, and better generalization accuracy - and dropout layers to decorrelate nodes.
model2 = keras.models.Sequential([
    Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(img_width,img_height,3)),
    BatchNormalization(),
    MaxPool2D(pool_size=(3,3), strides=(2,2)),
    Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    BatchNormalization(),
    MaxPool2D(pool_size=(3,3), strides=(2,2)),
    Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    BatchNormalization(),
    Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    BatchNormalization(),
    Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    BatchNormalization(),
    MaxPool2D(pool_size=(3,3), strides=(2,2)),
    Flatten(),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

#The AlexNet model uses SGD with learning rate of 0.001 as its optimizer
model2.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=SGD(learning_rate=0.001),
    metrics=['acc']    
)

#Model 3 (transfer learning): Uses a pretrained VGG-16 network, freezes the pretrained kayers to make them untrainable,
# and adds on classifcation layers at the end to suit our 4-class multiclassification problem.
pre_trained_model = VGG16(input_shape=(img_width, img_height, 3), include_top=False, weights="imagenet")
#freezing layers
for layer in pre_trained_model.layers:
  layer.trainable = False
last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output
x = GlobalMaxPool2D()(last_output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(4, activation='softmax')(x)

model3 = Model(pre_trained_model.input, x)
#This model uses Adam with learning rate of 0.001 as its optimizer
model3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])


## **Defining Regularization Components and Training the Four CNN Models**

We next move on to training all four models by fitting them to the training data and using the validation data thorughout the training process.

Importantly, we correct the previously mentioned class imbalances by computing class weights and using them to modify the loss function. With these weights, the model is penalized more for misclassifying examples from the minority classes. This helps to prevent the model from being biased towards the Normal majority class, which can occur if we do not use class weighting, as the model can mostly predict 'Normal' and achieve a high accuracy.

We employ learning rate reduction, which decreases the learning rate when the model's validation accuracy does not improve for two epochs. This helps tune the learning rate of the model, so that it converges to a local optimum and learns better.

To help avoid the model overfitting on the training data and resultingly not being able to generalize well to the held-out test data, we employ early stopping. This is a regularization technique for models trained using gradient descent, which stops the training process when the model's validation accuracy does not improve for 3 epochs. 

Note that the default batch size of 32 is used for all models. We wanted to keep many hyperparameters, such as batch_size, learning rate, and max number of epochs, the same across all four models. We wanted this to be a controlled experiment that would allow us to easily choose which of the four architectures could be selected as the 'best' and be kept for further hyperparameter tuning, interpretability analysis, and results.

In [ ]:
import numpy as np

#Defining early stopping and learning rate reduction objects for training
early = EarlyStopping(monitor="val_loss", mode="min", patience=3)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience = 2, verbose=1,factor=0.3, min_lr=0.000001)
callbacks_list = [ early, learning_rate_reduction ]

#weighting classes to account for minority classes (diseased) versus majority class (normal)
from sklearn.utils.class_weight import compute_class_weight
weights = compute_class_weight('balanced', np.unique(y_train), y_train)
cw = dict(zip( np.unique(y_train), weights))
print(cw)

#Train each model on training data and evaluate on test set.
#Also, get test data predictions for future confusion matrices

#define max number of epochs
n_epochs = 50

model0_history = model0.fit(X_train,y_train, epochs=n_epochs, validation_data=(X_val, y_val), class_weight=cw, callbacks=callbacks_list)
test_loss0, test_acc0 = model0.evaluate(X_test, y_test)
model0_preds = np.argmax(model0.predict(X_test), axis=-1)

model1_history = model1.fit(X_train,y_train, epochs=n_epochs, validation_data=(X_val, y_val), class_weight=cw, callbacks=callbacks_list)
test_loss1, test_acc1 = model1.evaluate(X_test, y_test)
model1_preds =np.argmax(model1.predict(X_test), axis=-1)


model2_history = model2.fit(X_train,y_train, epochs=n_epochs, validation_data=(X_val, y_val), class_weight=cw, callbacks=callbacks_list)
test_loss2, test_acc2 = model2.evaluate(X_test, y_test)
model2_preds = np.argmax(model2.predict(X_test), axis=-1)

model3_history = model3.fit(X_train,y_train, epochs=n_epochs, validation_data=(X_val, y_val), class_weight=cw, callbacks=callbacks_list)
test_loss3, test_acc3 = model3.evaluate(X_test, y_test)
model3_preds = np.argmax(model3.predict(X_test), axis=-1)


### **Training results from running locally:**
Note that most of our code had to be run locally and not on Google Colab due to memory constraints when using the server.

**Model 0**

  Epoch 1/50
484/484 [==============================] - 762s 2s/step - loss: 16.0766 - acc: 0.6394 - val_loss: 0.7649 - val_acc: 0.7282 - lr: 0.0010

Epoch 2/50
484/484 [==============================] - 809s 2s/step - loss: 0.6876 - acc: 0.7138 - val_loss: 0.5912 - val_acc: 0.7819 - lr: 0.0010

Epoch 3/50
484/484 [==============================] - 943s 2s/step - loss: 0.5323 - acc: 0.7718 - val_loss: 0.7870 - val_acc: 0.7028 - lr: 0.0010

Epoch 4/50
484/484 [==============================] - ETA: 0s - loss: 0.6092 - acc: 0.7523
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.

484/484 [==============================] - 1004s 2s/step - loss: 0.6092 - acc: 0.7523 - val_loss: 0.9394 - val_acc: 0.6537 - lr: 0.0010

Epoch 5/50
484/484 [==============================] - 993s 2s/step - loss: 0.5592 - acc: 0.7609 - val_loss: 0.6866 - val_acc: 0.7395 - lr: 3.0000e-04


**Model 1**

Epoch 1/50
484/484 [==============================] - 476s 981ms/step - loss: 2.3092 - acc: 0.5659 - val_loss: 0.7808 - val_acc: 0.6336 - lr: 0.0010

Epoch 2/50
484/484 [==============================] - 681s 1s/step - loss: 0.7119 - acc: 0.6644 - val_loss: 0.4773 - val_acc: 0.8047 - lr: 0.0010

Epoch 3/50
484/484 [==============================] - 580s 1s/step - loss: 0.5772 - acc: 0.7295 - val_loss: 0.4740 - val_acc: 0.8227 - lr: 0.0010

Epoch 4/50
484/484 [==============================] - 432s 893ms/step - loss: 0.4490 - acc: 0.7917 - val_loss: 0.3356 - val_acc: 0.8682 - lr: 0.0010

Epoch 5/50
484/484 [==============================] - 441s 910ms/step - loss: 0.4012 - acc: 0.8122 - val_loss: 0.4355 - val_acc: 0.8341 - lr: 0.0010

Epoch 6/50
484/484 [==============================] - ETA: 0s - loss: 0.3387 - acc: 0.8457
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.
484/484 [==============================] - 448s 925ms/step - loss: 0.3387 - acc: 0.8457 - val_loss: 0.3730 - val_acc: 0.8496 - lr: 0.0010

Epoch 7/50
484/484 [==============================] - 511s 1s/step - loss: 0.1986 - acc: 0.9007 - val_loss: 0.3248 - val_acc: 0.8842 - lr: 3.0000e-04

Epoch 8/50
484/484 [==============================] - 558s 1s/step - loss: 0.1551 - acc: 0.9209 - val_loss: 0.2664 - val_acc: 0.9173 - lr: 3.0000e-04

Epoch 9/50
484/484 [==============================] - 581s 1s/step - loss: 0.1262 - acc: 0.9368 - val_loss: 0.2556 - val_acc: 0.9147 - lr: 3.0000e-04

Epoch 10/50
484/484 [==============================] - 604s 1s/step - loss: 0.1082 - acc: 0.9441 - val_loss: 0.2845 - val_acc: 0.9132 - lr: 3.0000e-04

Epoch 11/50
484/484 [==============================] - ETA: 0s - loss: 0.0940 - acc: 0.9487
Epoch 11: ReduceLROnPlateau reducing learning rate to 9.000000427477062e-05.

Epoch 12/50
484/484 [==============================] - 583s 1s/step - loss: 0.0613 - acc: 0.9662 - val_loss: 0.3267 - val_acc: 0.9287 - lr: 9.0000e-05

**Model 2**

Epoch 1/50
484/484 [==============================] - 926s 2s/step - loss: 0.9763 - acc: 0.6121 - val_loss: 0.8580 - val_acc: 0.6217 - lr: 0.0010

Epoch 2/50
484/484 [==============================] - 1050s 2s/step - loss: 0.5563 - acc: 0.7195 - val_loss: 0.6063 - val_acc: 0.7514 - lr: 0.0010

Epoch 3/50
484/484 [==============================] - 1032s 2s/step - loss: 0.4279 - acc: 0.7651 - val_loss: 0.4736 - val_acc: 0.7933 - lr: 0.0010

Epoch 4/50
484/484 [==============================] - 907s 2s/step - loss: 0.3643 - acc: 0.7985 - val_loss: 0.4298 - val_acc: 0.8284 - lr: 0.0010

Epoch 5/50
484/484 [==============================] - 828s 2s/step - loss: 0.3271 - acc: 0.8158 - val_loss: 0.8187 - val_acc: 0.6641 - lr: 0.0010

Epoch 6/50
484/484 [==============================] - ETA: 0s - loss: 0.2882 - acc: 0.8384
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.
484/484 [==============================] - 653s 1s/step - loss: 0.2882 - acc: 0.8384 - val_loss: 0.4307 - val_acc: 0.8222 - lr: 0.0010

Epoch 7/50
484/484 [==============================] - 575s 1s/step - loss: 0.2505 - acc: 0.8586 - val_loss: 0.2578 - val_acc: 0.8997 - lr: 3.0000e-04

Epoch 8/50
484/484 [==============================] - 532s 1s/step - loss: 0.2249 - acc: 0.8652 - val_loss: 0.2901 - val_acc: 0.8941 - lr: 3.0000e-04

Epoch 9/50
484/484 [==============================] - ETA: 0s - loss: 0.2257 - acc: 0.8720
Epoch 9: ReduceLROnPlateau reducing learning rate to 9.000000427477062e-05.
484/484 [==============================] - 531s 1s/step - loss: 0.2257 - acc: 0.8720 - val_loss: 0.2603 - val_acc: 0.8997 - lr: 3.0000e-04

Epoch 10/50
484/484 [==============================] - 530s 1s/step - loss: 0.2085 - acc: 0.8763 - val_loss: 0.2588 - val_acc: 0.9044 - lr: 9.0000e-05

**Model 3**

Epoch 1/50
484/484 [==============================] - 2493s 5s/step - loss: 3.1034 - acc: 0.6480 - val_loss: 0.5986 - val_acc: 0.7364 - lr: 0.0010

Epoch 2/50
484/484 [==============================] - 2311s 5s/step - loss: 0.5876 - acc: 0.7153 - val_loss: 0.4869 - val_acc: 0.8134 - lr: 0.0010

Epoch 3/50
484/484 [==============================] - 2306s 5s/step - loss: 0.4851 - acc: 0.7463 - val_loss: 0.4635 - val_acc: 0.8460 - lr: 0.0010

Epoch 4/50
484/484 [==============================] - 2325s 5s/step - loss: 0.4717 - acc: 0.7517 - val_loss: 0.3773 - val_acc: 0.8584 - lr: 0.0010

Epoch 5/50
484/484 [==============================] - 2303s 5s/step - loss: 0.4464 - acc: 0.7660 - val_loss: 0.4538 - val_acc: 0.8160 - lr: 0.0010

Epoch 6/50
484/484 [==============================] - ETA: 0s - loss: 0.3989 - acc: 0.7919
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.
484/484 [==============================] - 2309s 5s/step - loss: 0.3989 - acc: 0.7919 - val_loss: 0.4330 - val_acc: 0.8429 - lr: 0.0010

Epoch 7/50
484/484 [==============================] - 2313s 5s/step - loss: 0.3102 - acc: 0.8330 - val_loss: 0.2607 - val_acc: 0.9018 - lr: 3.0000e-04

Epoch 8/50
484/484 [==============================] - 2271s 5s/step - loss: 0.2735 - acc: 0.8469 - val_loss: 0.3004 - val_acc: 0.8904 - lr: 3.0000e-04

Epoch 9/50
484/484 [==============================] - ETA: 0s - loss: 0.2523 - acc: 0.8586 
Epoch 9: ReduceLROnPlateau reducing learning rate to 9.000000427477062e-05.
484/484 [==============================] - 5626s 12s/step - loss: 0.2523 - acc: 0.8586 - val_loss: 0.2640 - val_acc: 0.8951 - lr: 3.0000e-04

Epoch 10/50
484/484 [==============================] - 5117s 11s/step - loss: 0.2257 - acc: 0.8707 - val_loss: 0.2175 - val_acc: 0.9152 - lr: 9.0000e-05

Epoch 11/50
...

Epoch 13/50
484/484 [==============================] - 2296s 5s/step - loss: 0.1986 - acc: 0.8863 - val_loss: 0.2311 - val_acc: 0.9127 - lr: 2.7000e-05


<a name = 'comparing'></a>
## **Calling Evaluation Functions to See How Each Model Performed**
After training all four CNNs and getting their test set predictions, we call pur pre-defined evaluation functions for each model. Using these results, we can select the best architecture to use for further hyperparameter tuning and analysis.

In [ ]:
#model 0
print(test_acc0, test_loss0)
plot_confusion_matrix(y_test, model0_preds)
generate_classifcation_report(y_test, model0_preds, 2)
plot_accuracy(model0_history)
plot_loss(model0_history)

**test accuracy:** 0.7479338645935059

**average recall:** 0.7250660396363505
**average precision:** 0.6099109638940649

**F-beta score (beta = 2):** 0.69865718799

**classifcation report:**

                         precision    recall  f1-score   support

      NORMAL             0.96      0.78      0.86      1372
      COVID              0.54      0.60      0.57       351
      TUBERCULOSIS       0.17      0.68      0.27        68
      PNEUMONIA          0.77      0.84      0.80       145
      accuracy                               0.75      1936
      macro avg          0.61      0.73      0.63      1936
      weighted avg       0.84      0.75      0.78      1936

![picture](https://drive.google.com/uc?export=view&id=1iPCA_0ll3t14yhrFGFjrqn1d8fFmlLS6)
![picture](https://drive.google.com/uc?export=view&id=1QLdg6cqwoSIKeu80S79URUFJ7XZGd18j)
![picture](https://drive.google.com/uc?export=view&id=1kvkWgtwEd6RpSAar5CERUIzj2MH04w_Z)




In [ ]:
#model 1
print(test_acc1, test_loss1)
plot_confusion_matrix(y_test, model1_preds)
generate_classifcation_report(y_test, model1_preds, 2)
plot_accuracy(model1_history)
plot_loss(model1_history)

**test accuracy:** 0.9266529083251953

**average recall:** 0.89904468144885
**average precision:** 0.8958676404549528

**F-beta score (beta = 2):** 0.90040688575

**classifcation report:**

                       precision    recall  f1-score   support

       NORMAL            0.97      0.94      0.95      1372
       COVID             0.80      0.90      0.84       351
       TUBERCULOSIS      0.88      0.85      0.87        68
       PNEUMONIA         0.94      0.90      0.92       145
       accuracy                               0.93      1936
       macro avg          0.90      0.90      0.90      1936
       weighted avg       0.93      0.93      0.93      1936


![picture](https://drive.google.com/uc?export=view&id=1W4FTApNm5XfGqt7CRYqJXYSrgf_EwyLc)
![picture](https://drive.google.com/uc?export=view&id=1DFOztJ2TSo_uE5fIZmg8_JmGOVHW6gPu)
![picture](https://drive.google.com/uc?export=view&id=1kGKI9sZu_a1FqgwYymlmu9i068sVehlG)



In [ ]:
#model 2
print(test_acc2, test_loss2)
plot_confusion_matrix(y_test, model2_preds)
generate_classifcation_report(y_test, model2_preds, 2)
plot_accuracy(model2_history)
plot_loss(model2_history)

**test accuracy:** 0.89462810754776

**average recall:** 0.9343007758027214
**average precision:** 0.8223037395216595

**F-beta score (beta = 2):**  0.909223117

**classification report:**

                          precision    recall  f1-score   support

       NORMAL             0.98      0.88      0.93      1372
       COVID              0.71      0.92      0.80       351
       TUBERCULOSIS       0.80      0.97      0.87        68
       PNEUMONIA          0.80      0.97      0.88       145
       accuracy                               0.89      1936
       macro avg          0.82      0.93      0.87      1936
       weighted avg       0.91      0.89      0.90      1936

![picture](https://drive.google.com/uc?export=view&id=1SALFcvnHwB4_3Lj0kcG_QqCg5EN0-gWP)
![picture](https://drive.google.com/uc?export=view&id=1umH568yJvvmtjgTlxE2SeyyavqINc9Nz)
![picture](https://drive.google.com/uc?export=view&id=19QNG0m3q7hCjg2I0dCx2M7vdp5nMtdk0)

In [ ]:
#model 3
print(test_acc3, test_loss3)
plot_confusion_matrix(y_test, model3_preds)
generate_classifcation_report(y_test, model3_preds, 2)
plot_accuracy(model3_history)
plot_loss(model3_history)

**test accuracy:** 0.9085744023323059

**average recall:** 0.9250757742783864
**average precision:** 0.8510064392931764 

**F-beta score (beta = 2):** 0.90918803418

**classification report:**

                  precision    recall  f1-score   support

      NORMAL       0.97      0.90      0.94      1372
      COVID        0.76      0.91      0.83       351
      TUBERCULOSIS 0.84      0.93      0.88       68
      PNEUMONIA    0.83      0.97      0.89      145
      accuracy                         0.91      1936
      macro avg    0.85      0.93      0.88      1936
      weighted avg 0.92      0.91      0.91      1936

![picture](https://drive.google.com/uc?export=view&id=19yrPXtWCZs1oYxFP_lFr7AqtjQHLsmYf)
![picture](https://drive.google.com/uc?export=view&id=1ekTbIXPHvQwLPLfDp4tf8DD5_K5L5isX)
![picture](https://drive.google.com/uc?export=view&id=1RuN2BGcziTGlthLO1OqpwjKIvgLl1IoN)

## **Summary of Results, Selection of Best Model, and Reasoning**

Models 1, 2, and 3 all outperformed the baseline Model 0 in terms of accuracy, average recall, average precision, and F-beta score when evaluated on the held-out test data. The baseline model had a substantially lower accuracy of 74.8%, while the rest had over 90%. More importantly, the baseline had substantially lower average recall and precision than the other three models. The three deep models had very similar, notably high F-beta scores, recall, precision, and accuracies.

Thus, we see there is a clear benefit of adding more depth and complexity to the CNN architecture, as well as the benefit of dropout and batch normalization layers. Having deeper structures allows the CNN to extract more complex and abstract features in the Chest X-rays, which improves their ability to recognize distinguishing patterns among the four classes.

 

Looking at the confusion matrices generated for each model, we see all four models were least confused with chest X-rays labeled Normal and most confused with the chest X-rays labeled with Tuberculosis. This makes sense since the classes are very imbalanced, with the most frequent class being Normal and least frequent class being Tuberculosis. Notice that employing class weighting to correct this imbalance during model training seemed to work, as all four models had similar precision and recall across the four classes and did not substantially suffer on any one class. However, since most training examples were labeled Normal, it makes sense that each model is least confused by this class and is able to correctly classify it the most, given that it has seen far more Normal examples than any other class.

Looking at the learning curves (loss and accuracy) for each model, we see:

> Model 0: The baseline model had a fairly unstable learning curve, and the validation accuracy quickly plateaued/only increased very slighlty for a few epochs in a row, triggering early stopping. This indicates the shallow model's poor ability to recognize patterns in the data to aid classifcation.

> Model 1: This model has a slighlty unstable learning curve but perfomed much better than the baseline. We begin to see some overfit around epoch 8, as validation accuracy plateaus and validation loss begins to increase again while training loss continues to decrease. Early stopping prevented further overfit to the training data.

> Model 2: The AlexNet model has a notably unstable learning curve. We begin to see some slight overfit around epoch 8, as validation accuracy plateaus and validation loss begins to increase again while training loss continues to decrease. 

> Model 3: The VGG-16 has a slighlty unstable learning curve. We begin to see some overfit around epoch 10, as validation accuracy begins to slightly decrease and validation loss begins to slightly increase while training loss continues to decrease.

###**Selection of Best Model:**

Since Model 1 had the highest test accuracy - which is a reasonable metric to use given that we weighted the 4 classes to account for imbalances in the data - has very similar recall/precison/F-beta score as Models 2 & 3, and **also took the most reasonable amount of time to train**, we'll carry on with Model 1 as our best model for further hyperparameter tuning, visualizations, and interpretability analysis.

One surprising aspect is why Model 3 did not perform substantially better than the other 3 models, given that we used a pre-trained, high-performing, deep network. We discuss some possible explanations in our results section.

<a name = 'hyperparametertuning'></a>
#**Hyperparameter Tuning of Best Model (Model 1)**

To further improve the accuracy, precision, and recall of the best model, Model 1, we performed hyperparameter tuning to find the best learning rate, number of epochs, and number of units in the dense layer. As noted above, the learning curve for this model was slightly unstable, so our second aim with this tuning process is to smooth out the learning curve and acheive a more stable model.

These two hyperparameters were tuned using keras tuner, which has options for Hyperband, Bayesian Optimization, GridSearch, and Random search. Tuning learning rate is important because we do not want to end up with unstable training with a large learning rate or a learning rate that is too small and takes too long to converge. The number of units in the first dense layer is important as it represents the dimensionality of the output. Tuning other layers would be optimal, however, due to the limits of our computational resources and time constraints, we decided just to tune these two. 

When we were trying to stabilize the learning curve of the best model (later on), we found that the model was particularly sensitive to minibatch size and learning rate. Reducing the learning rate from 0.0001 to 0.00005 significantly increased the instability of the learning curve, and increasing the batch size from 64 to 128 substantially weakened model performance.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

# Uses Model1 - which we found to be the best out of our baseline models
def model_builder(hp):
    
    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)

    # Tune learning rate for Adam optimizer with values from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
    
    model1 = Sequential([
      Conv2D(16,(3,3),activation = "relu" , input_shape = (img_width,img_height,3)) ,
      MaxPooling2D(2,2),
      Conv2D(32,(3,3),activation = "relu") ,  
      MaxPooling2D(2,2),
      Conv2D(64,(3,3),activation = "relu") ,  
      MaxPooling2D(2,2),
      Conv2D(128,(3,3),activation = "relu"),  
      MaxPooling2D(2,2),
      Flatten(), 
      Dense(hp_units,activation="relu"),      #Adding the Hidden layer
      Dropout(0.1,seed = 2019),
      Dense(400,activation ="relu"),
      Dropout(0.3,seed = 2019),
      Dense(300,activation="relu"),
      Dropout(0.4,seed = 2019),
      Dense(200,activation ="relu"),
      Dropout(0.2,seed = 2019),
      Dense(4,activation = "softmax")   #Adding the Output Layer
    ])
    
    # Define optimizer, loss, and metrics
    model1.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(),
                  metrics=["acc"])
    
    return model1

tuner = kt.Hyperband(model_builder,
                     objective="val_acc",
                     max_epochs=20,
                     factor=3,
                     hyperband_iterations=10,
                     directory="kt_dir_2",
                     project_name="kt_hyperband",)


# Display search space summary
tuner.search_space_summary()

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early], verbose=2)

# Get the optimal hyperparameters from the results
best_hps=tuner.get_best_hyperparameters()[0]
print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(X_train,y_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_acc']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(X_train,y_train, epochs=best_epoch, validation_split=0.2)

eval_result = hypermodel.evaluate(X_test, y_test)
print("[test loss, test accuracy]:", eval_result)

ModuleNotFoundError: ignored

##**Best Hyperparameters**
**Note**: Due to time constraints, we had to stop the hyperparameter tuning process early (we let it run for ~19 hours) and take the best values at that moment, listed here.


1.   Learning rate: 0.0001
2.   Number of units in first densely connected layer: 512

Next, we modify Model 1 and retrain it to incorporate these optimal hyperparamter values.



In [ ]:
#constructing and training best model (no early stopping or reduction of learning rate)
from keras import backend as K
best_lr = 0.0001
best_n_epochs = 10
best_dense_units = 512
best_model = Sequential([
    Conv2D(16,(3,3),activation = "relu" , input_shape = (img_width,img_height,3)) ,
    MaxPooling2D(2,2),
    Conv2D(32,(3,3),activation = "relu") ,  
    MaxPooling2D(2,2),
    Conv2D(64,(3,3),activation = "relu") ,  
    MaxPooling2D(2,2),
    Conv2D(128,(3,3),activation = "relu"),  
    MaxPooling2D(2,2),
    Flatten(), 
    Dense(best_dense_units,activation="relu"),      #Adding the Hidden layer
    Dropout(0.1,seed = 2019),
    Dense(400,activation ="relu"),
    Dropout(0.3,seed = 2019),
    Dense(300,activation="relu"),
    Dropout(0.4,seed = 2019),
    Dense(200,activation ="relu"),
    Dropout(0.2,seed = 2019),
    Dense(4,activation = "softmax")   #Adding the Output Layer
])

best_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics = ['acc'])
K.set_value(best_model.optimizer.learning_rate, best_lr)

best_history = best_model.fit(X_train,y_train, epochs=best_n_epochs, validation_data=(X_val, y_val), class_weight=cw)
test_loss, test_acc = best_model.evaluate(X_test, y_test)
best_model_preds = np.argmax(best_model.predict(X_test), axis=-1)

**output from running locally**

Epoch 1/10 484/484 [==============================] - 755s 2s/step - loss: 1.4392 - acc: 0.6255 - val_loss: 0.4944 - val_acc: 0.8155 Epoch 2/10 484/484 [==============================] - 573s 1s/step - loss: 0.4739 - acc: 0.7775 - val_loss: 0.3582 - val_acc: 0.8661 Epoch 3/10 484/484 [==============================] - 519s 1s/step - loss: 0.3362 - acc: 0.8341 - val_loss: 0.3248 - val_acc: 0.8822 Epoch 4/10 484/484 [==============================] - 497s 1s/step - loss: 0.2550 - acc: 0.8757 - val_loss: 0.6073 - val_acc: 0.8078 Epoch 5/10 484/484 [==============================] - 480s 991ms/step - loss: 0.2280 - acc: 0.8914 - val_loss: 0.6024 - val_acc: 0.8264 Epoch 6/10 484/484 [==============================] - 488s 1s/step - loss: 0.1902 - acc: 0.9064 - val_loss: 0.3043 - val_acc: 0.8915 Epoch 7/10 484/484 [==============================] - 476s 983ms/step - loss: 0.1368 - acc: 0.9289 - val_loss: 0.2146 - val_acc: 0.9261 Epoch 8/10 484/484 [==============================] - 473s 977ms/step - loss: 0.0973 - acc: 0.9467 - val_loss: 0.2264 - val_acc: 0.9261 Epoch 9/10 484/484 [==============================] - 484s 999ms/step - loss: 0.1010 - acc: 0.9498 - val_loss: 0.1790 - val_acc: 0.9571 Epoch 10/10 484/484 [==============================] - 548s 1s/step - loss: 0.0875 - acc: 0.9564 - val_loss: 0.2352 - val_acc: 0.9318

We next call our evluation functions on the model to see how hyperparameter tuning improved model performance.

In [ ]:
print(test_acc, test_loss)
plot_confusion_matrix(y_test, best_model_preds)
generate_classifcation_report(y_test, best_model_preds, 2)
plot_accuracy(best_model_history)
plot_loss(best_model_history)


**test accuracy:** 0.9318181872367859

**avg recall:** 0.9494431581521567

**avg precision:** 0.8509724214764968

**F-beta score:** 0.92763496439

                        precision    recall  f1-score   support

      NORMAL             0.99      0.92      0.96      1408
      COVID              0.80      0.95      0.87       335
      TUBERCULOSIS       0.76      0.97      0.85        64
      PNEUMONIA          0.85      0.95      0.90       129
      accuracy                               0.93      1936
      macro avg          0.85      0.95      0.89      1936
      weighted avg       0.94      0.93      0.93      1936



![picture](https://drive.google.com/uc?export=view&id=1QGmin8CoZdOQWOAX134c2yxR7EHgpE_H)
![picture](https://drive.google.com/uc?export=view&id=1ILIlm7hwtsHxYO787kLUCyEUlqhLSYS6)
![picture](https://drive.google.com/uc?export=view&id=13YGuDqlx5_9SvIavwZxZj07eSk4a7K0S)

Compared to the evaluation results of Model 1 before doing hyperparameter tuning, we see tuning these values and using their optimal values slightly improved model performancce and ability to correctly classify the chest X-rays. The accuracy increased slightly from 92.7% to 93.2%, the average recall substantially increased from 0.899 to 0.949, as did the F-2 score, increasing from 0.900 to 0.928. We see the average precision did decrease from 0.896 to 0.851. In addition, based off the confusion matrix, this model is slightly less confused by the minority classes Tuberculosis and Pneumonia. However, as discussed earlier, the real-life implication of low recall is more severe than low precision, so we conclude that the hyperparameter tuned model is better overall.

However, as is evident from the plots above and was pointed out by Professor Liu, the learning curves exhibit unstable behavior. We experimented with different combinations of increased minibatch sizes (from the default size  of 32) and decreased learning rates to try to smooth out and stabilize the loss/accuracy curves. 

After a few different trials, in which the model was particularly sensitive to the learning rate and suffered from a smaller rate, we found a minibatch size of 64 with the same learning rate of 0.0001 yielded the most stable learning curves. We retrained the model and evaluate it here:

In [ ]:
best_history = best_model.fit(X_train,y_train, epochs=best_n_epochs, validation_data=(X_val, y_val), class_weight=cw, batch_size = 64)
test_loss, test_acc = best_model.evaluate(X_test, y_test)
best_model_preds = np.argmax(best_model.predict(X_test), axis=-1)

**results from running locally**

Epoch 1/10
242/242 [==============================] - 483s 2s/step - loss: 1.6489 - acc: 0.5615 - val_loss: 0.5468 - val_acc: 0.7804
Epoch 2/10
242/242 [==============================] - 403s 2s/step - loss: 0.5345 - acc: 0.7465 - val_loss: 0.4197 - val_acc: 0.8279
Epoch 3/10
242/242 [==============================] - 593s 2s/step - loss: 0.3714 - acc: 0.8159 - val_loss: 0.2879 - val_acc: 0.8837
Epoch 4/10
242/242 [==============================] - 463s 2s/step - loss: 0.2844 - acc: 0.8512 - val_loss: 0.4743 - val_acc: 0.8072
Epoch 5/10
242/242 [==============================] - 417s 2s/step - loss: 0.2168 - acc: 0.8851 - val_loss: 0.2145 - val_acc: 0.9142
Epoch 6/10
242/242 [==============================] - 514s 2s/step - loss: 0.1882 - acc: 0.9041 - val_loss: 0.1956 - val_acc: 0.9313
Epoch 7/10
242/242 [==============================] - 402s 2s/step - loss: 0.1538 - acc: 0.9207 - val_loss: 0.1684 - val_acc: 0.9375
Epoch 8/10
242/242 [==============================] - 572s 2s/step - loss: 0.1176 - acc: 0.9419 - val_loss: 0.1503 - val_acc: 0.9457
Epoch 9/10
242/242 [==============================] - 441s 2s/step - loss: 0.0937 - acc: 0.9510 - val_loss: 0.1254 - val_acc: 0.9556
Epoch 10/10
242/242 [==============================] - 501s 2s/step - loss: 0.0798 - acc: 0.9580 - val_loss: 0.1583 - val_acc: 0.9416

**Evaluation results**

**test accuracy:** 0.9457644820213318

**avg recall:** 0.9427777351645651

**avg precision:**0.9134578429111846

**F-beta score:** 0.9368433

                        precision    recall  f1-score   support

      NORMAL             0.98      0.95      0.96      1368
       COVID             0.85      0.94      0.90       358
      TUBERCULOSIS       0.90      0.99      0.94        72
      PNEUMONIA          0.92      0.89      0.91       138
      accuracy                               0.95      1936
      macro avg          0.91      0.94      0.93      1936
      weighted avg       0.95      0.95      0.95      1936


![picture](https://drive.google.com/uc?export=view&id=1fl3EJa8ZYfMIlllKirMFQCJPN3F4rdkd)
![picture](https://drive.google.com/uc?export=view&id=11OQC-KKB6yF3yWHoAUtWwWG-NRMxgX_j)
![picture](https://drive.google.com/uc?export=view&id=1m4Bl89YAlPqdH2g_jbqj0VdgwGNrA_yt)

As we see, with the exception of the jump at epoch 5 (which a similar problem occured in every trial/combination of batch size and learning rate), the learning curves are much smoother than before, and the model's accuracy, precision, and F-2 score increased, while the recall is almost the same. Yay!

We will carry on the interpretability analysis, comparison to classical models, further data experiments, and results with this as our best model.

<a name = 'histeq'></a>

## **Using Histogram Equalization on Data and Seeing How Performance of Best CNN Model Changes**

As a further experiment/deliverable, we wanted to explore using histogram equalization to process our image data and retraining our best model on that transformed data to see how performance changes. 

Histogram equalization is an image processing technique that essential manipulates an image to widen the range of pixel intensities, rather than have all intensities concentrated at one value. This improves the overall global contrast of the image. Here is the code that performs this processing technique on all the image data, as well as a before and after example of how the process changes an example image.

In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

#performing histogram equalization on all train, val, test
X_train_eq = []
for img_arr in X_train:
  img = Image.fromarray(img_arr)
  grayimg = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)
  eq = cv2.equalizeHist(np.array(grayimg))
  eq_arr = cv2.resize(eq,(img_D,img_D))
  X_train_eq.append(eq_arr)

X_val_eq = []
for img_arr in X_val:
  img = Image.fromarray(img_arr)
  grayimg = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)
  eq = cv2.equalizeHist(np.array(grayimg))
  eq_arr = cv2.resize(eq,(img_D,img_D))
  X_val_eq.append(eq_arr)

X_test_eq = []
for img_arr in X_test:
  img = Image.fromarray(img_arr)
  grayimg = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)
  eq = cv2.equalizeHist(np.array(grayimg))
  eq_arr = cv2.resize(eq,(img_D,img_D))
  X_test_eq.append(eq_arr)

#testing to see how it worked
uneq_img = Image.fromarray(np.uint8(X_val[0]))
display(uneq_img)
eq_img = Image.fromarray(np.uint8(X_val_eq[0]))
display(eq_img)

![picture](https://drive.google.com/uc?export=view&id=1AroY0DvlkXy1J49qsFlQcsfgMuk20UF5)

![picture](https://drive.google.com/uc?export=view&id=1EHP9A0PTJU0P_2xC-ge0JsAS6UfY7th1)

We next retrain our best model, this time using the transformed train and validation data. Our hope is that by training on the improved cotrast images, the features/anomalies that indicate the presence of different diseases will be more apparrent. Thus, the model may be able to better extract the features and patterns that differentiate the four classes and perform better classification of the chest X-rays.

We then evaluated the fitted model on the transformed test data.

In [ ]:
#retraining best CNN model using transformed data and comparing results

#train on equalized data
best_model_history = best_model.fit(X_train_eq, y_train, validation_data=(X_val_eq, y_val), class_weight = cw, epochs = best_n_epochs)

In [ ]:
#evaluate on equalized test data
test_loss_eq, test_acc_eq = best_model.evaluate(X_test_eq, y_test)
preds_eq = np.argmax(best_model.predict(X_test_eq), axis=-1)
generate_classifcation_report(y_test, preds_eq, 2)
plot_confusion_matrix(y_test, preds_eq)
plot_accuracy(best_model_history)
plot_loss(best_model_history)

**results from running locally**

Epoch 1/10
484/484 [==============================] - 976s 2s/step - loss: 0.5661 - acc: 0.7436 - val_loss: 0.5938 - val_acc: 0.7628
Epoch 2/10
484/484 [==============================] - 1990s 4s/step - loss: 0.3532 - acc: 0.8278 - val_loss: 0.2869 - val_acc: 0.8848
Epoch 3/10
484/484 [==============================] - 1162s 2s/step - loss: 0.2565 - acc: 0.8729 - val_loss: 0.2690 - val_acc: 0.8946
Epoch 4/10
484/484 [==============================] - 747s 2s/step - loss: 0.1831 - acc: 0.9100 - val_loss: 0.3158 - val_acc: 0.8941
Epoch 5/10
484/484 [==============================] - 995s 2s/step - loss: 0.1229 - acc: 0.9367 - val_loss: 0.1852 - val_acc: 0.9323
Epoch 6/10
484/484 [==============================] - 1312s 3s/step - loss: 0.1098 - acc: 0.9421 - val_loss: 0.1947 - val_acc: 0.9328
Epoch 7/10
484/484 [==============================] - 666s 1s/step - loss: 0.0809 - acc: 0.9579 - val_loss: 0.2026 - val_acc: 0.9406
Epoch 8/10
484/484 [==============================] - 673s 1s/step - loss: 0.0548 - acc: 0.9723 - val_loss: 0.3070 - val_acc: 0.9142
Epoch 9/10
484/484 [==============================] - 648s 1s/step - loss: 0.0460 - acc: 0.9747 - val_loss: 0.2194 - val_acc: 0.9426
Epoch 10/10
484/484 [==============================] - 624s 1s/step - loss: 0.0256 - acc: 0.9842 - val_loss: 0.2100 - val_acc: 0.9504


**Evaluation results**

**test accuracy:** 0.9545454382896423

**avg recall:** 0.9374846088663448

**avg precision:**0.9182584599230237

**F-beta score:** 0.9341374078

                      precision    recall  f1-score   support

      NORMAL             0.98      0.97      0.97      1408
       COVID             0.90      0.92      0.91       335
      TUBERCULOSIS       0.92      0.92      0.92        64
      PNEUMONIA          0.88      0.95      0.91       129
      accuracy                               0.95      1936
      macro avg          0.92      0.94      0.93      1936
      weighted avg       0.96      0.95      0.95      1936

![picture](https://drive.google.com/uc?export=view&id=1_ck0zc0UrNSofbvAeykKvO4HPFzDw1DV)
![picture](https://drive.google.com/uc?export=view&id=1UVrIlNU3YQCNUUgDyb3-dtHHyI-bnEqc)
![picture](https://drive.google.com/uc?export=view&id=1p9omezF8whi3HTyYFD7APIHhMH3v9AKt)

We see that while some evaluation metrics - accuracy and precision - improved slightly, these increases were not nearly as substantial as we expected, given that the model was trained on improved contrast data. In addition, recall and the F-2 score dropped slightly. As discussed, recall is a high-priority metric for us. In addition, we see from the loss curve that this learning curve is fairly unstable.

We concluded that performing histogram equalization did not yield a better-performing model and was not a worthwhile data transformation to improve classification results.

<a name = 'expanddata'></a>
## **Expansion of Data Set to Include Other Diseases and Data Augmentation**

As a further deliverable, we wanted to expand our dataset to include chest X-rays labeled with other types of diseases besides Normal, Tuberculosis, Pneumonia, or COVID-19. We wanted to see if our best model would perform similarly well on being able to differetniate between more types of diseases and be able to correctly classify among a more diverse set of categories. In terms of real-life implications, this has a direct practical use, as being able to identify many different types of diseases is useful for providing more types of diagnoses.

In addition, as mentioned, our classes are extremely imbalanced. We want to try a form of oversampling from the minority classes via data augmentation of the least frequent classes in our dataset (LIST). This will provide more training examples from these classes, and it may help our model with improved recall and precision for those classes and further prevent bias towards the majority class. Again, there are important real-life implications: since Normal is our majority class, it is very dangerous to have a model that is biased towards predicting Normal in order to acheive a high accuracy. This can lead to undiagnosed disease, so sick people do not get the treatment they need.

First, we expanded our dataset to include chest X-rays labeled with general, Non-Covid lung infections, known as "lung opacity" in the dataset. After adding these images we reshuffled the dataset, and made new train/test/val splits with an 80-10-10 distribution.

In [ ]:
# expanding dataset to include chest x ray scans of patients with lung opacity - non covid related lung diseases
lung_op_path="lung-scans-dataset/lung-opacity" # 299 x 299 

# read in covid images and store in X, y arrays
lung_op_X = []
lung_op_y = [] #4

for img in os.listdir(lung_op_path):
    image_path=lung_op_path+"/"+img
    img_arr=cv2.resize(cv2.imread(image_path),(img_D,img_D))
    lung_op_X.append(img_arr)
    lung_op_y.append(4)
    
# intialize new expanded X and y dataset
X_exp = np.concatenate((covid_X, normal_X, tuber_X, pneum_X, lung_op_X), axis=0)
y_exp = np.concatenate((covid_y, normal_y, tuber_y, pneum_y, lung_op_y), axis=0)

# randomize all data
X_exp, y_exp = shuffle(X_exp, y_exp, random_state=0)

# create train/test/val split
from sklearn.model_selection import train_test_split
# creates a train set made of 80% of the data
X_exp_train, X_exp_test, y_exp_train, y_exp_test = train_test_split(X_exp, y_exp, test_size=0.20)
# splits the previous test set into test and val evenly, so each is 10% of the data
X_exp_val, X_exp_test, y_exp_val, y_exp_test = train_test_split(X_exp_test, y_exp_test, test_size=0.50)

Here, we provide an example of the ImageDataGenerator 

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from os import listdir
from PIL import Image

# example data gen
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')


# preview some images of datagen
i = 0
for batch in datagen.flow(covid_X[0], batch_size=1, save_to_dir='preview', save_prefix='covid', save_format='png'):
    i += 1
    if i > 20:
        break 

preview_path = "preview"

for img in os.listdir(preview_path):
    image_path=lung_op_path+"/"+img
    Image.open(image_path).show()

In [ ]:
# recconstructing best model to fit expanded dataset (from 4 to 5 classes)
# (no early stopping or reduction of learning rate)
from keras import backend as K
best_lr = 0.0001
best_n_epochs = 10
best_dense_units = 512
best_model = Sequential([
    Conv2D(16,(3,3),activation = "relu" , input_shape = (img_width,img_height,3)) ,
    MaxPooling2D(2,2),
    Conv2D(32,(3,3),activation = "relu") ,  
    MaxPooling2D(2,2),
    Conv2D(64,(3,3),activation = "relu") ,  
    MaxPooling2D(2,2),
    Conv2D(128,(3,3),activation = "relu"),  
    MaxPooling2D(2,2),
    Flatten(), 
    Dense(best_dense_units,activation="relu"),      #Adding the Hidden layer
    Dropout(0.1,seed = 2019),
    Dense(400,activation ="relu"),
    Dropout(0.3,seed = 2019),
    Dense(300,activation="relu"),
    Dropout(0.4,seed = 2019),
    Dense(200,activation ="relu"),
    Dropout(0.2,seed = 2019),
    Dense(5,activation = "softmax")   #Adding the Output Layer
])

best_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics = ['acc'])
K.set_value(best_model.optimizer.learning_rate, best_lr)

In [ ]:
batch_size = 64 # from previous hyperparameter tuning

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(X_exp_train, y_exp_train, batch_size=batch_size)

validation_generator = test_datagen.flow(X_exp_val, y_exp_val, batch_size=8)

best_history = best_model.fit(
    train_generator, 
    validation_data=validation_generator, 
    epochs=best_n_epochs, 
    steps_per_epoch=len(X_exp_train) / batch_size,
    class_weight=cw)

test_loss, test_acc = best_model.evaluate(X_exp_test, y_exp_test)
best_model_preds = np.argmax(best_model.predict(X_exp_test), axis=-1)

In [ ]:
print(test_acc, test_loss)
plot_confusion_matrix(y_test, best_model_preds)
generate_classifcation_report(y_test, best_model_preds, 2)
plot_accuracy(best_model_history)
plot_loss(best_model_history)

**Evaluation Results:**

**Test Accuracy:**

**Average Recall:**

**Average Precision:**

**F-2 Score:**

**Classifcation Report**:





<a name = 'classicalmodels'></a>
## **Comparison of Best CNN with SVM and Random Forest Classifier Models for Image Classification**

Before discussing final results and the interpretability of our best model, we wanted to perform a final experiment. As a last deliverable, we are training an SVM and Random Forest Classifer on the image data and comparing their evaluation to our best CNN model.

We want to see how these more classical methods fare compared to our deep learning method to see if there is a substantial benefit to using deep learning for image classification tasks. In the era of deep learning, CNNs are the go-to method for this task, as discussed in the beginning of this report. This comparison of models allows us to examine if there is a notable advantage to the more complex deep learning models for performing medical imaging, multiple class classifcation tasks.

We first performed GridSearch on the C, gamma, and kernel type hyperparameters for the SVM and RFC classifiers. Note that due to training time constraints (these models took exceptionally long to train on our large datasets and images), we had to limit the number of different values used for these parameters.

In [ ]:

from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# defining and training SVM classifier
#using GridSearchCV to find optimal parameters

#defining parameter grid
param_grid={'C':[0.1,1],'gamma':[0.001,1],'kernel':['rbf','poly']}

#defining classifier with class weights to correct imbalances
svc=svm.SVC(probability=True, class_weight= cw)

#defining grid search model and fitting to training data to get optimal parameters
svm_cv=GridSearchCV(svc,param_grid)
svm_cv.fit(X_train, y_train)
print(svm_cv.best_params_) 



**best SVM parameters:** C = 0.01, gamma = 0.01, kernel = 'rbf'


In [ ]:
# defining and training decision tree classifier
#using GridSearchCV to find optimal parameters

from sklearn.ensemble import RandomForestClassifier

#parameter grid
param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt'],
    'max_depth' : [4,6, 8],
    'criterion' :['gini']
}

#defining classifier with class weights to correct imbalances
rfc=RandomForestClassifier(class_weight = cw)

#defining grid search model and fitting to training data to get optimal parameters
rfc_cv = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
rfc_cv.fit(X_train, y_train)
print(rfc_cv.best_params_)

**best RFC parameters:** max_features='auto', n_estimators= 200, max_depth=8, criterion='gini'

Using these optimal hyperparameter values, we trained our SVM classifier on the original training and validation sets, using class weights to correct for the class imbalances. However, thhese intial training sessions resulted in two very poor classifiers: The SVM classifier only predicted Normal on the test data, and the RFC classifier had low precision and recall on all classes except Normal.

Despite using class weightings, training these classifiers on imbalanced data resulted in very poor performance. SVMs especially are known to suffer with imbalanced data, and they often will only predict the majority class in these cases in order to maximize accuracy, as we saw.

In an effort to fix this class imbalance problem, we undersampled from the majority Normal and COVID-19 classes by removing 90% of the Normal examples and 67% of the COVID-19 examples. This resulted in a fairly balanced training set. Here is the code that performs this undersampling and prints the fairly balanced class weights that result:

In [ ]:
#reshaping data since SVM and RFC takes 2-dimensional arrays as input
X_train_d2 = X_train.reshape((X_train.shape[0],img_D*img_D*3))
X_small = X_train_d2
y_small = y_train

#removing 90% of Normal examples and 67% of Covid examples
ind0 = np.where(y_small == 0)[0]
ind0 = ind0[0:9*int(len(ind0) / 10)]
X_small = X_small.tolist()

for i in sorted(ind0, reverse=True):
    elem = X_small[i]
    X_small.remove(elem)
    y_small = np.delete(y_small, i)

ind1 = np.where(y_small == 1)[0]
ind1 = ind1[0:2*int(len(ind1) / 3)]
     
for i in sorted(ind1, reverse=True):
    elem = X_small[i]
    X_small.remove(elem)
    y_small = np.delete(y_small, i)

weights = compute_class_weight(class_weight = 'balanced', classes = np.unique(y_small), y = y_small)
cw = dict(zip( np.unique(y_small), weights))

**fairly balanced class weights:** {0: 0.8446524064171123, 1: 0.9642857142857143, 2: 1.5763473053892216, 3: 0.8736172566371682}

Using this more balanced training set, we refit the SVM and RFC classifiers and evaluated them on the test set. Note that the loss function for the SVM classifer is squared hinge loss, Python's default loss function for its SVM classifier. The loss function for the Random Forest classifier is gini, which measures the quality of a tree split (in terms of information gain).

In [ ]:
#training SVM with optimal parameters and getting predictions/accuracies

svm_model = svm.SVC(probability=True, class_weight= cw, C = 0.01, gamma = 0.01, kernel = 'rbf')
svm_model.fit(X_small, y_small)

In [ ]:
#evaluating the fitted SVM classifier
X_testd2 = X_test.reshape((X_test.shape[0],img_D*img_D*3))
svm_pred= svm_model.predict(X_testd2)
print(f"The model is {accuracy_score(svm_pred,y_test)*100}% accurate")
plot_confusion_matrix(y_test, svm_pred)
generate_classification_report(y_test, svm_pred, 2)

**Evaluation results**

**test accuracy:** 0.72

**avg recall:** 0.18

**avg precision:**0.25

**F-beta score:** 0.191

                        precision    recall  f1-score   support

      NORMAL             0.72      1.00      0.84      1390
       COVID             0.00      0.00      0.00       356
      TUBERCULOSIS       0.00      0.00      0.00        60
      PNEUMONIA          0.00      0.00      0.00       130
      accuracy                               0.72      1936
      macro avg          0.18      0.25      0.21      1936
      weighted avg       0.52      0.72      0.60      1936

![picture](https://drive.google.com/uc?export=view&id=1tp1Bw_QP8yJacWyM8pVAcboADeqniiAy)


In [ ]:
#training RFC with optimal parameters and getting predictions/accuracies
rfc_model=RandomForestClassifier(class_weight = cw, max_features='auto', n_estimators= 200, max_depth=8, criterion='gini')
rfc_model.fit(X_small, y_small)

In [ ]:
rfc_pred= svm_model.predict(X_testd2)
print(f"The model is {accuracy_score(rfc_pred,y_test)*100}% accurate")
plot_confusion_matrix(y_test, rfc_pred)
generate_classification_report(y_test, rfc_pred, 2)

**Evaluation results**

**test accuracy:** 0.8161

**avg recall:** 0.847

**avg precision:**0.685

**F-beta score:** 0.809

                      precision    recall  f1-score   support

      NORMAL             0.96      0.81      0.88      1365
       COVID             0.65      0.77      0.71       379
      TUBERCULOSIS       0.63      0.84      0.72        63
      PNEUMONIA          0.49      0.96      0.65       129
      accuracy                               0.82      1936
      macro avg          0.69      0.85      0.74      1936
      weighted avg       0.86      0.82      0.83      1936

![picture](https://drive.google.com/uc?export=view&id=1C3kc3Wer4435jTcR3Z-vaDTvBiGNgSlk)

### **Discussion of SVM and RFC Performance**

We see from the confusion matrix generated from the SVM's test predictions that the model only predicted one class for all test cases - the majority class, Normal. Thus, applying class weighting and downsampling from the majority classes to balance out our training data still did not help the model learn useful features for the classification task. As discussed, imabalanced classes are a huge issue for SVM classifiers that can really hinder its performance. One approach that would likely help the model learn most would be to do feature engineering to extract useful features from each image as a form of represenation learning, and pass that list of features to the SVM to train on, rather than the raw image data. This would be a good experiment if we had more time, but unfortunately we simply ran out of time and were not able to.

Compared to the SVM, the Random Forest classifier performed much better and was able to learn at least some useful features for differentiating between the four classes and classifying images somewhat well. However, we see it was still significantly confused on the Tuberculosis and Pneumonia examples,  and it did not acheive nearly as high recall, accuracy, or precision as our best CNN model.

Thus, our best CNN performed substantially better than both classical models. We conclude that using deep learning is extremely valuable for image classifcation tasks, as these networks can use spatial relations and local information in images to extract useful, abstract features to aid in classifying chest X-rays.

<a name = 'interpret'></a>
# **Discussion of Interpretability of Best CNN Model**

We've seen from the high accuracy, precision, and recall thar our best CNN model performs well on this image classifcation task and is successful with being able to correctly classify and predict the presence of different diseases (or no disease) based on chest X-ray scans. However, related to our unit on bias and fairness and human-centered ML, we also want our model to be interpretable. We want to know *why* the network classifies one scan as Tuberculosis and one as Normal, as this gives some insight into what sort of features/patterns the model is trying to detect. Knowing this, medical professionals and related experts can assess if the model is recognizing the correct sort of features for detecting these diseases, and if there seems to be any present bias in its predictions.

A huge limitation of deep learning and neural networks is that their predictions and outputs are hard to explain and interpret, due to the blackbox nature of their complex structures. One advantage of CNNs that operate on image data is that we can visualize the model's internal workings through feature maps, filters, and heatmaps produced with software like GradCam. These methods help provide insight on which parts of the image the model is attending to and uses most for its predictions. We'll implement a few techniques for understanding and visualizing the innerworking of our best CNN, to hopefully make its decision processes more interpretable by humans.

## **Best Model Architecture**
We'll first print the model architecture in order to visualize its system of input, hidden, and output layers.

In [ ]:
print(best_model.summary())

## **Model: "sequential_1"**
_________________________________________________________________
**Layer (type)                Output Shape              Param #** 
------------------------------------------------------------------

 conv2d_4 (Conv2D)           (None, 297, 297, 16)      448       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 148, 148, 16)     0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 146, 146, 32)      4640      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 35, 35, 64)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 33, 33, 128)       73856     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 16, 16, 128)      0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 32768)             0         
                                                                 
 dense_5 (Dense)             (None, 512)               16777728  
                                                                 
 dropout_4 (Dropout)         (None, 512)               0         
                                                                 
 dense_6 (Dense)             (None, 400)               205200    
                                                                 
 dropout_5 (Dropout)         (None, 400)               0         
                                                                 
 dense_7 (Dense)             (None, 300)               120300    
                                                                 
 dropout_6 (Dropout)         (None, 300)               0         
                                                                 
 dense_8 (Dense)             (None, 200)               60200     
                                                                 
 dropout_7 (Dropout)         (None, 200)               0         
                                                                 
 dense_9 (Dense)             (None, 4)                 804       
                                                                 
--------------------------------------------------------------
Total params: 17,261,672

Trainable params: 17,261,672

Non-trainable params: 0
_________________________________________________________________

<a name = 'featuremaps'></a>
## **Visualizing Feature Maps of Best Performing CNN Model**

Feature maps are generated by applying the learned filters at each layer to the input image/X-ray scan. These help understand the learned features explicitly, as we can visualize what sort of shapes/features the model is extracting at each hidden layer, whether it be simple lines/edges, parts of the chest, or the chest as a whole.

In [ ]:
# visualizing filters for early hidden layers

from tensorflow.keras.utils import load_img, img_to_array

layer_names = [layer.name for layer in best_model.layers]
layer_outputs = [layer.output for layer in best_model.layers]
feature_map_model = Model(best_model.input, layer_outputs)

image_path= "C:/Users/Owner/Desktop/lung-scans-dataset/lung-scans-dataset/tuberculosis/Tuberculosis-5.png"
img = load_img(image_path, target_size=(299, 299))  
input = img_to_array(img)                          
input = input.reshape((1,) + input.shape)                   
input /= 299.0

plt.figure()
plt.imshow(img, aspect='auto')

feature_maps = feature_map_model.predict(input)

for layer_name, feature_map in zip(layer_names, feature_maps):print(f"The shape of the {layer_name} is =======>> {feature_map.shape}")


for layer_name, feature_map in zip(layer_names, feature_maps): 
   if len(feature_map.shape) == 4:
      print(layer_name)
      k = feature_map.shape[-1]  
      size=feature_map.shape[1]
      for i in range(k):
        feature_image = feature_map[0, :, :, i]
        feature_image-= feature_image.mean()
        feature_image/= feature_image.std ()
        feature_image*=  64
        feature_image+= 128
        plt.figure()
        plt.imshow( feature_image, aspect='auto')

**Input Image:**

![picture](https://drive.google.com/uc?export=view&id=1c4j51IrtyfYxyq63w17fO2tBCiKnk2IQ)

**A few examples of the feature maps produced:**

![picture](https://drive.google.com/uc?export=view&id=1IvCWrt0t0bZKMqnckv9LZp54YcpOHYyI)
![picture](https://drive.google.com/uc?export=view&id=1kFkRpMtbjVBSX0G1VYat4wcquW_Rm2hJ)
![picture](https://drive.google.com/uc?export=view&id=1AwRxitfsJRRYDIYssW5C9r96CrtwqmLK)
![picture](https://drive.google.com/uc?export=view&id=17ppp15zLUV3GEyqsOhXr8lXVi4k48X25)
![picture](https://drive.google.com/uc?export=view&id=11egSKaqdn085apE20M4ZUUNImkMa-vH1)
![picture](https://drive.google.com/uc?export=view&id=1YLXU_Zjlcn5RQ0WCkPRzhMfb8r4n551V)

We see that at the earlier convolution layers, the model is extracting simpler, low-level features like general shapes, edges, or parts like the spine itself. As we progress to deeper layers, more abstract and high-level features can be extracted, such as the lungs themselves or X-ray as a whole. This gives insight into how the model is building up its recognition of different features, and the abstract features of the chest X-rays that are helping it most in discerning between classes and predicting the most probably category for the image. 

For example, we see the model pays particular attention to the overall skeletal shape, the central region/spine, the lungs and patterns present within them, and large spots that may be present in the lungs. This mix of low and high level features aids the model's classification abilities.

## **Visualizing Heatmaps with GradCam**

In this technique, we use GradCAM and try to understand which parts of the input chest X-ray led the CNN to a particular classification decision. This technique is called “Class Activation Map” (CAM) visualization and consists of producing heatmaps of “class activation” over input images. This heatmap is a 2D grid of scores associated with a specific output class, computed for every location in an input image, indicating how important each location is with respect to the class considered. 

Relating to our descriptions of what certain patterns are present for the different disease types, this may help provide insight on which parts of the X-rays led our model to classifying a scan as having Tuberculosis versus being Normal, etc.

In [ ]:
from tensorflow import GradientTape, reduce_mean, newaxis, squeeze, maximum
from tensorflow.math import reduce_max
from tensorflow.keras.applications.imagenet_utils import decode_predictions

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = np.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., newaxis]
    heatmap = squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = maximum(heatmap, 0) / reduce_max(heatmap)
    return heatmap.numpy()

from tensorflow.keras.utils import array_to_img, img_to_array
import matplotlib.cm as cm
from IPython.display import Image, display
def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.4):
    # Load the original image
    img = load_img(img_path)
    img = img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    display(Image(cam_path))


# Prepare image

img_path= "C:/Users/Owner/Desktop/lung-scans-dataset/lung-scans-dataset/normal-1/Normal-9.png"
img_arr=cv2.resize(cv2.imread(img_path),(299,299))
print(np.array([img_arr]).shape)
preds = best_model.predict(np.array([img_arr]))
print(preds)

layer_names = [layer.name for layer in best_model.layers]
print(layer_names)
last_conv_layer_name = 'conv2d_27'

# Generate class activation heatmap
heatmap = make_gradcam_heatmap(np.array([img_arr]), best_model, last_conv_layer_name)
save_and_display_gradcam(img_path, heatmap)

Here are a few example visuals produced for each of the four classes by using our model to make a prediction and obtaining the CAM heatmap. We see in all four cases, the model was extremely sure of its (correct) prediciton, as the class probabilit of the correct class was close to 1 in all four cases:

>> Tuberculosis:

>>>![picture](https://drive.google.com/uc?export=view&id=1LB149jyV6mZUAWAkS5du8lfjkvo9YFX6)

>>> Prediction Probabilities: [[4.8890292e-05 3.5538847e-05 9.9988961e-01 2.5968213e-05]]

>>> Beyond the heatmap, we can see the presence of some white, dense regiona of infections towards the center of the chest/lungs, which as explained earlier is a common feature of Tuberculosis-infected lungs. We see, in addition to focusing on some points on the perimeter of the lungs, the model is attending to these center spots. Thus, the model may have learned that these center-focused spots are associated with the presence of TB, helping it in its predicitons. 

>> Pneumonia: 

>>>![picture](https://drive.google.com/uc?export=view&id=1M2CIpJkIbor3RAPkYhoEs3cRdmYqU_0q)

>>>In this example, the model seems to be attending to many spots all ob=ver the lungs. This may relate to the pattern of white spots on lungs infected with pneumonia that we discussed at the beginning of the report. The model may have learned to recognize these spots and associate them with pneumonia.

>>> Prediction Probabilities: [2.7230291e-12 2.3518005e-14 1.5038323e-14 1.0000000e+00]]

>> Covid: 

>>>![picture](https://drive.google.com/uc?export=view&id=1Lc57YZUWH9dWs9hZkT56hYFNE6wu6K6U)


>>> Prediction Probabilities: [[1.15993455e-04 9.97398496e-01 1.84514327e-04 2.30111461e-03]]

>>> The model is mostly focusing on the perimeter of the lungs, indicating it may detect some areas of infection/inflammation on the surrounding of the lungs. It may have learned to associate surrounding infections with Covid, aiding it in its predictions.

>> Normal:

>>>![picture](https://drive.google.com/uc?export=view&id=1VhL9ogkIkZxkeYkuzW_FBDb01KV0wUF1)


>>> Prediction Probabilities: [[9.9996626e-01 2.5662697e-05 7.4023683e-06 7.2378020e-07]]

  >>> We see that there does not appear to be any particular area/pattern that the model focuses on in the X-ray. This may indicate that, since these lungs are healthy, the model inteprets Normal lungs as having no discernable anomlalies, which helps it to make its predicitons. The model may have learned what patterns indicate the presence of other diseases, so having none of these patetrns indicates the lungs are Normal.

## **Plotting One vs Rest AUC-ROC Curves**
As a final analysis step, we plotted one vs Rest AUC (Area Under The Curve) ROC (Receiver Operating Characteristics) curves for the various classes. This is a performance measurement for classification problems at various threshold settings. ROC is a probability curve and AUC represents the degree or measure of separability. It tells how much the model is capable of distinguishing between classes.  An excellent model has AUC near to the 1 which means it has a good measure of separability. Ideally, the true negative and true positive curves should not overlap at all, and the model is perfectly able to distinguish between the positive (one) and negative (rest) classes.

We plot AUC-ROC curves for each of the four classes based on our model's test predicitons. This allows us to easily visualize if the model can successfully distinguish between all four classes and is not particularly biased/substantially better than discerning one class versus all the others. This helps to ensure the fairness of our model.

In [ ]:
from sklearn.preprocessing import LabelBinarizer

label_binarizer = LabelBinarizer().fit(y_train)
y_onehot_test = label_binarizer.transform(y_test)
y_onehot_test.shape  # (n_samples, n_classes)

#y_score = best_model.predict(X_test)
class_of_interest = 2
class_id = np.flatnonzero(label_binarizer.classes_ == class_of_interest)[0]
class_id

import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay

RocCurveDisplay.from_predictions(
    y_onehot_test[:, class_id],
    y_score[:, class_id],
    name=f"{class_of_interest} vs the rest",
    color="darkorange",
)
plt.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves: Tuberculosis vs Rest")
plt.legend()
plt.show()

 
![picture](https://drive.google.com/uc?export=view&id=1XRskjOGVcE6Yc_HuSdvUYJdZHxw7f0Yk)
 ![picture](https://drive.google.com/uc?export=view&id=1uvfiezVFXJJbH5igcG9zwgCd5BwHyiP8)
  ![picture](https://drive.google.com/uc?export=view&id=18NEF61kv3hgVt3Y7IAoUOQmbAdRLdbDa)
  ![picture](https://drive.google.com/uc?export=view&id=1EpNEakFSsa5QSj_xWQGEmjSloSSBKBB4)

 
 As we can see, all four plots have AUC close to 1 and have the ideal shape for an excellent degree of seperability, indicating our model can successfully distinguish each of the four classes form the rest based on the chest X-rays.

# **Results**
                          Baseline   Model 1     Model 2      Model 3          SVM    RFC    Best Model
             Acc.          .748       .932         .895          .909         .72     .8161    0.946
             Precision.    .610       .851         .822          .851         .18     .847    0.913
             Recall.       .725       .949         .934          .825         .25     .685    0.943
             F2 Score.     .699       .928         .909          .909         .191    .809    0.937

  Throught the evaluation and comparison of the baseline CNN, Model 1 (deeper CNN), AlexNet implementation, and transfer learning model with VGG-16, we decided Model 1 was our best model due to it having the highest accuracy, similar precision/recall/F-2 score compared to Models 2 and 3, and the most reasonable amount of time taken to train. When we tuned the hyperparameters of this model, we found the most optimal parameters were ⁉

  >> Learning rate: 0.0001

  >> Number of epochs: 10

  >>Number of units in first dense layer: 512

  >> Batch Size: 64

  In comparison to the CNN models, the SVM and RF classifiers performed substantially worse, especially the SVM classsifier. This is likely due to the SVM's poor ability to learn on the raw image data, and some feature engineering/extraction would need to be performed.


**What about these results surprised you? Why?**
-TODO: add more!

One surprising result VGG-16 didn't perform as well as expected given we used a pre-trained, well-performing deep model. Given that this model has been trained on thousands of images and has achieved very good performance, we would've expected the model to perform substantially better than the shallower Model 1 or AlexNet architecture (Model 2). One potential explanation is, given the depth of this model, there were much more parameters to learn. As we know from lecture, increasing the depth of neural networks tends to increase chances of overfit. Based on the plots of validation loss as a function of number of epochs for Model 3, there seems to be some slight overfit occuring as validation loss begins to increase as training loss continues to decrease:

![picture](https://drive.google.com/uc?export=view&id=1ekTbIXPHvQwLPLfDp4tf8DD5_K5L5isX)

This indicates some slight overfit began to occur, so the depth of this architecture may have been a limiting factor in being able to outperform the other models.

In addition, we applied transfer learning on a pretrained VGG-16 model to use for our classifcation task. One limitation of transfer learning we found through our research was that it will not work well when the high-level features learned by the bottom layersof the pretrained model are not sufficient to differentiate the classes in your problem. It may be the case that the pretrained model was not able to extract the most useful features for differntiating between the four categories in the chest X-rays.
 
**Did your models over- or under-fit? How can you tell? What did you do to address these issues?**

Here is the plot of validation and training accuracy as a function of number of epochs for our best model (Model 1) when its hyperparameters, including batch size, were tuned:

  ![picture](https://drive.google.com/uc?export=view&id=11OQC-KKB6yF3yWHoAUtWwWG-NRMxgX_j)

From the plot, we see some very slight overfit at the end when validation accuracy slightly drops off as training accuracy continues to increase. However, we employed several regularization techniques to help prevent our model from overfitting significantly on the training data, including adding dropout layers to the architecture to decorrelate nodes and make it more generalizable, and employing early stopping while training. Also, we did hyperparameter tuning on the number of epochs to find the best number of iterations, since overfit on the training data can occur when the number of epochs is too high.

**What does the evaluation of your trained models tell you about your data?** 

All CNN modelsperformed pretty well and were able to classify the chest X-ray correctly according to their labelbed disease or no disease in most cases, while the SVM and RFC performed substantially worse. this tells us our raw image data - without any feature engineering/extraction performed - is well suited to be learnable by CNNs. For the CNN Models (1,2,3) they had recalss of .949, .934, and .825 respectively. This indicates a low false negative rate and the CNNs ability to find relevant cases from the dataset.

# **Discussion**

## What you've learned

**What concepts from lecture/breakout were most relevant to your project? How so?**
Our project was most relevant to class lecture topics Deep learning, CNNs, and fairness and evaluation metrics since we trained and tested the deep cnn models to classify x-ray images. We then evaluated the performance using accuracy, recall, precision, F-beta score.

**What lessons did you take from this project that you want to remember for the next ML project you work on? Do you think those lessons would transfer to other datasets and/or models? Why or why not?**

One interesting lesson that we can take away from the project is learning the whole pipeline of how to process and use raw data, to train and test a model, and then evaluate them. These lessons might transfer to other datasets and models but would be slightly different depending on the specific model and its parameters. 

**What was the most helpful feedback you received during your presentation? Why?**
One helpful suggestions was Professor Liu's suggestion to further our discussion of model intepretability to strengthen our project. The interpretability of deep learning models is extremely valuable for real-life applications, especially for ones like ours that relate to medical diagnoses.

Another helpful piece was classmates' suggestions to expand on explanation of the different model architectures, why we think VGG-16 may not have performed as well, etc. Follwoing these suggestions these helped to build a more cohesive and well-explained report.

**If you had two more weeks to work on this project, what would you do next? Why?**

We would perform feature engineering/extraction on the raw image data to use as training data for the SVM and Random Forest classifiers. As we saw, they performed substantially worse than the CNN models when trained on the raw image data, especially the SVM model. With more time, we could extract more useful features that are better suited for these models to learn on, and then we could more fairly compare these classical models to deep learning methods. 

# **References (Datasets, Relevant Research, and Example Code/Tutorials)**

Tawsifur Rahman, Amith Khandakar, Muhammad A. Kadir, Khandaker R. Islam, Khandaker F. Islam, Zaid B. Mahbub, Mohamed Arselene Ayari, Muhammad E. H. Chowdhury. (2020) "Reliable Tuberculosis Detection using Chest X-ray with Deep Learning, Segmentation and Visualization". IEEE Access, Vol. 8, pp 191586 - 191601. DOI. 10.1109/ACCESS.2020.3031384.

“TB Incidence in the U.S.” Centers for Disease Control and Prevention, Centers for Disease Control and Prevention, 5 Oct. 2021, https://www.cdc.gov/tb/statistics/tbcases.htm. 

“FastStats - Pneumonia.” Centers for Disease Control and Prevention, Centers for Disease Control and Prevention, 6 Sept. 2022, https://www.cdc.gov/nchs/fastats/pneumonia.htm. 

M.E.H. Chowdhury, T. Rahman, A. Khandakar, R. Mazhar, M.A. Kadir, Z.B. Mahbub, K.R. Islam, M.S. Khan, A. Iqbal, N. Al-Emadi, M.B.I. Reaz, M. T. Islam, “Can AI help in screening Viral and COVID-19 pneumonia?” IEEE Access, Vol. 8, 2020, pp. 132665 - 132676.

Rahman, T., Khandakar, A., Qiblawey, Y., Tahir, A., Kiranyaz, S., Kashem, S.B.A., Islam, M.T., Maadeed, S.A., Zughaier, S.M., Khan, M.S. and Chowdhury, M.E., 2020. Exploring the Effect of Image Enhancement Techniques on COVID-19 Detection using Chest X-ray Images.

https://clinmedjournals.org/articles/jfmdp/journal-of-family-medicine-and-disease-prevention-jfmdp-4-073.php?jid=jfmdp

https://www.radiologyinfo.org/en/info/pneumonia#:~:text=When%20interpreting%20the%20x%2Dray,(fluid%20surrounding%20the%20lungs).

https://nyulangone.org/news/truth-about-covid-lung#:~:text=Early%20on%2C%20hazy%20areas%20of,sufficient%20for%20diagnosing%20COVID%2D19.

https://www.analyticsvidhya.com/blog/2021/03/introduction-to-the-architecture-of-alexnet/#:~:text=The%20Alexnet%20has%20eight%20layers,layers%20except%20the%20output%20layer.

https://www.geeksforgeeks.org/vgg-16-cnn-model/

https://towardsdatascience.com/transfer-learning-with-convolutional-neural-networks-in-pytorch-dd09190245ce#:~:text=The%20basic%20premise%20of%20transfer,layers%20which%20make%20a%20prediction.

“Convolutional Neural Network (CNN).” TensorFlow, https://www.tensorflow.org/tutorials/images/cnn.

Alake, Richmond. “Implementing AlexNet CNN Architecture Using TensorFlow 2.0+ and Keras.” Towards Data Science, 14 Aug. 2020, https://towardsdatascience.com/implementing-alexnet-cnn-architecture-using-tensorflow-2-0-and-keras-2113e090ad98.

Munawar, M. R. (2022, May 26). Image classification using transfer learning (VGG-16). Medium. https://medium.com/nerd-for-tech/image-classification-using-transfer-learning-vgg-16-2dc2221be34c 

Sudhakar, S. (2021, January 30). Histogram equalization. Medium. Retrieved December 8, 2022, from https://towardsdatascience.com/histogram-equalization-5d1013626e64 

Chollet, François. Keras. 2.11.0, Apache 2.0, 14 November 2022.

R. R. Selvaraju, M. Cogswell, A. Das, R. Vedantam, D. Parikh and D. Batra. Grad-CAM. IEEE. 2017.

Keras Tuner - TensorFlow Tutorial (https://www.tensorflow.org/tutorials/keras/keras_tuner)

Cournapeau, David. Scikit-Learn. 1.1.3 New BSD license, June 2007
Intel. OpenCV. 4.6.0, Apache license, June 2000.

https://towardsdatascience.com/understanding-auc-roc-curve-68b2303cc9c5

https://neptune.ai/blog/transfer-learning-guide-examples-for-images-and-text-in-keras

https://towardsdatascience.com/visual-interpretability-for-convolutional-neural-networks-2453856210ce

https://towardsdatascience.com/understanding-auc-roc-curve-68b2303cc9c5

https://www.analyticsvidhya.com/blog/2020/11/tutorial-how-to-visualize-feature-maps-directly-from-cnn-layers/

https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html